In [1]:
from qaravan.core import * 
from qaravan.tensorQ import *

In [2]:
# testing QubitNoise

t1 = 1000
t2 = 1000
one_qubit_time = 10
two_qubit_time = 100

nm = QubitNoise(t1, t2, one_qubit_time, two_qubit_time)
kraus_list = nm.get_kraus(time=two_qubit_time) 

sum([k.conj().T @ k for k in kraus_list])

array([[1., 0.],
       [0., 1.]])

In [12]:
# testing PauliNoise

strings = ['ii', 'ix', 'zy', 'zz']
probs = [0.9, 0.04, 0.03, 0.03]
nm = PauliNoise(strings, probs)
kraus_list = nm.get_kraus()
sum([k.conj().T @ k for k in kraus_list])

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [20]:
strings = ['ii', 'xx', 'yx']
probs = [0.3, 0.4, 0.3]
nm = PauliNoise(strings, probs)

nm_inv = nm.pinv()
nm_mod = nm_inv.compose(nm)
print(nm_mod)

{'ii': np.float64(1.0)}


In [21]:
ptm     = nm.ptm()                 # diagonal matrix F
ptm_inv = nm_inv.ptm()             # F^{-1}
print(np.max(np.abs(ptm_inv @ ptm - np.eye(ptm.shape[0]))))

2.220446049250313e-16


In [22]:
S      = nm.get_superop()
S_inv  = nm_inv.get_superop()
print(np.max(np.abs(S_inv @ S - np.eye(S.shape[0]))))

3.15


In [9]:
# testing PauliLindbladNoise

p1 = 0.3
p2 = 0.4

strings = ['xx', 'yx']
probs = [p1, p2]
channel = PauliLindbladNoise(strings, probs)
superop = channel.get_superop().reshape([2]*8)
print(channel.rates)

dm = random_dm(2)
state = dm.reshape([2]*4)
gate_indices, state_indices = generate_gate_indices([0,1], 2), generate_state_indices([0,1], 2)
new_state = ncon((superop, state), (gate_indices, state_indices))
new_dm = new_state.reshape(4,4)

r = channel.rates
P0, r0 = embed_operator(2, [0,1], [pauli_mapping[s] for s in strings[0]], dense=True), channel.rates[0]
P1, r1 = embed_operator(2, [0,1], [pauli_mapping[s] for s in strings[1]], dense=True), channel.rates[1]

dm_a = (1-r0)*dm + r0*P0 @ dm @ P0.T.conj()
dm_b = (1-r1)*dm_a + r1*P1 @ dm_a @ P1.T.conj()
np.allclose(dm_b, new_dm)

[np.float64(0.22559418195298675), np.float64(0.2753355179413892)]


np.complex128(0.2487653780640991+0j)

In [8]:
# testing sampling from PL channels 

true_freqs = [(1-r0)*(1-r1), (1-r0)*r1, r0*(1-r1), r0*r1]
samples = [channel.sample_string() for _ in range(10000)]

from collections import Counter
counts = Counter(samples)
keys = ['ii', 'yx', 'xx', 'zi']
freqs = [counts[k] / 10000 for k in keys]

for i in range(4):
    print(f"True freq: {true_freqs[i]}, Sampled freq: {freqs[i]}")
    assert np.isclose(true_freqs[i], freqs[i], atol=0.05)

True freq: 0.5611843910382136, Sampled freq: 0.5525
True freq: 0.21322142700879962, Sampled freq: 0.2207
True freq: 0.16348009102039715, Sampled freq: 0.1653
True freq: 0.062114090932589607, Sampled freq: 0.0615


In [ ]:
# test manual simulation of PL channel

gate_list = [H(0), RZ(0, np.pi/4), H(0)]
circ = Circuit(gate_list, 2)

mat = circ.to_matrix()
init_sv = np.array([1, 0, 0, 0])
final_sv = mat @ init_sv
dm = np.outer(final_sv, final_sv.conj())

op = (np.kron(pauli_Z, pauli_I) + np.kron(pauli_I, pauli_Z))/(2*np.sqrt(2))
noiseless_exp = np.trace(op @ dm).real

strengths = np.logspace(-6, 0, num=100)
noisy_exp_list = []
for strength in strengths: 
    coeffs = np.random.rand(2) * strength
    strings = ['xx', 'yx']
    channel = PauliLindbladNoise(strings, coeffs)
    superop = channel.get_superop().reshape([2]*8)
    #print(channel.rates)

    state = dm.reshape([2]*4)
    gate_indices, state_indices = generate_gate_indices([0,1], 2), generate_state_indices([0,1], 2)
    new_state = ncon((superop, state), (gate_indices, state_indices))
    noisy_dm = new_state.reshape(4,4)

    noisy_exp_list.append(np.trace(op @ noisy_dm).real)

plt.plot(strengths, noisy_exp_list, label='noisy expectation')
plt.axhline(y=noiseless_exp, color='r', linestyle='--', label='noiseless expectation')
plt.xscale('log')
plt.xlabel('Pauli-Lindblad noise strength')
plt.ylabel('magnetization')
plt.legend()

In [ ]:
# testing DensityMatrixSim with PL noise

gate_list = [H(0), RZ(0, np.pi/4), H(0)]
circ = Circuit(gate_list, 2)

mat = circ.to_matrix()
init_sv = np.array([1, 0, 0, 0])
final_sv = mat @ init_sv
dm = np.outer(final_sv, final_sv.conj())

op = (np.kron(pauli_Z, pauli_I) + np.kron(pauli_I, pauli_Z))/(2*np.sqrt(2))
noiseless_exp = np.trace(op @ dm).real

strengths = np.logspace(-6, 0, num=100)
noisy_exp_list = []
for strength in strengths: 
    coeffs = np.random.rand(2) * strength
    strings = ['xx', 'yx']
    channel = PauliLindbladNoise(strings, coeffs)

    sim = DensityMatrixSim(circ=copy.deepcopy(circ), nm=channel)
    sim.run(progress_bar=False)
    dm = sim.get_density_matrix()
    noisy_exp = np.trace(op @ dm).real
    noisy_exp_list.append(noisy_exp)

plt.plot(strengths, noisy_exp_list, label='noisy expectation')
plt.axhline(y=noiseless_exp, color='r', linestyle='--', label='noiseless expectation')
plt.xscale('log')
plt.xlabel('Pauli-Lindblad noise strength')
plt.ylabel('magnetization')
plt.legend()